## Imports


In [14]:
import numpy as np
import rasterio
import subprocess
import matplotlib.pyplot as plt
from io import BytesIO
import matplotlib.colors as mcolors
from pystac_client import Client
from IPython.display import Image
from PIL import Image as pil_image

## Fetch Landsat Scene from Washington D.C. Area

In [2]:
# Connect to STAC API and search for Landsat 9 imagery
catalog = Client.open("http://ec2-54-172-212-55.compute-1.amazonaws.com/api/v1/pgstac/")
bbox = [-76.964657, 38.978967, -76.928008, 39.002783]

search = catalog.search(
    collections=["landsat-c2l1"],
    bbox=bbox,
)

items = search.get_all_items()
len(items)

1

## Fetch bands 4 and 5 from the scene (Red and NIR)

In [3]:
item = items[0]
# Get red and NIR band assets and access alternate keys
red_band_cid = item.assets["red"].extra_fields["alternate"]["IPFS"]["href"].split("/")[-1]
nir_band_cid = item.assets["nir08"].extra_fields["alternate"]["IPFS"]["href"].split("/")[-1]

print(f"Red band CID: {red_band_cid}")
print(f"NIR band CID: {nir_band_cid}")

Red band CID: QmTgttqUf7PvZgdSoe71j3njeEKk1hC3h22n2sQmety3To
NIR band CID: QmZkWaKSuVhFKtAwNbxSogcT6hXHMksXjhgqLu6AXHSUKq


## Helper Functions

In [4]:
def load_landsat_band(band: bytes) -> np.ndarray:
    with rasterio.MemoryFile(band) as memfile:
        with memfile.open() as dataset:
            return dataset.read(1)


def save_plot_to_buffer(plot: plt.Figure) -> bytes:
    buffer = BytesIO()
    plot.savefig(buffer, format='png')
    buffer.seek(0)
    return buffer.getvalue()

## Fetch Bands from IPFS

In [5]:
red_band = subprocess.check_output(["ipfs", "cat", red_band_cid])
nir_band = subprocess.check_output(["ipfs", "cat", nir_band_cid])

 82.62 MiB / 82.62 MiB  100.00% 0s
 89.73 MiB / 89.73 MiB  100.00% 0sK


## Load the bands into a numpy array

In [7]:
red_band_4 = load_landsat_band(red_band)
nir_band_5 = load_landsat_band(nir_band)

## Calculate NDVI

In [8]:
eps = 0.0001 # Avoid divide by zero errors
ndvi = (nir_band_5 - red_band_4) / (nir_band_5 + red_band_4 + eps)

## Plot NDVI

In [ ]:
# Set min and max values for better color differentiation
ndvi_min, ndvi_max = -1, 1

# Create a custom color map
cmap = plt.cm.RdYlGn
norm = mcolors.Normalize(vmin=ndvi_min, vmax=ndvi_max)

# Plot the NDVI image
fig, ax = plt.subplots()
im = ax.imshow(ndvi, cmap=cmap, norm=norm)
cbar = fig.colorbar(im, ax=ax, label='NDVI', cmap=cmap, norm=norm)
ax.set_title('Normalized Difference Vegetation Index (NDVI)')

## Save plot to image (png) buffer

In [10]:
# Save the plot to a BytesIO buffer
plot_buffer = save_plot_to_buffer(fig)

## Add plot image to IPFS

In [12]:
# Add the image buffer to IPFS and get the hash. The -q flag is used to only return the hash of the added file
ipfs_hash = subprocess.check_output(["ipfs", "add", "-q"], input=plot_buffer).decode().strip()
ipfs_hash

'QmZasRR5ath2hAXApYKwy4droFhiHAPpk4yvBDuKvKRYGE'

## Fetch plot image from IPFS and display it

In [ ]:
ipfs_plot_png = subprocess.check_output(["ipfs", "cat", ipfs_hash])
img = pil_image.open(BytesIO(ipfs_plot_png))
display(img)